# Extract Toxicity Network Data

*This workbook repacks the results objects from the IDTxl estimation process (4_estimate.py) into arrays suitable for network analysis.*

- **INPUTS:** the following input files must be in the data directory:
    - `resampled.csv` (resampled toxicity timeseries data, output of 4_resample.py)
    - `[OUTPUT_FILE]_results_iterations_[iterations]`.pkl (mTE estimation results, output of 5_estimate.py)
    - `[OUTPUT_FILE]_globalID_iterations_[iterations]` (global edge identifiers, output of 5_estimate.py)
    - `[OUTPUT_FILE]_localID_iterations_[iterations]` (local to global edge conversions, output of 5_estimate.py)

## setup

In [ ]:
#GENERAL
import os
import pickle
import itertools
import numpy as np
import pandas as pd

#data directory:
datadir = os.getcwd()+'/../../data/'

#declare filenames:
resampled_filename = 'resampled.csv' #resampled toxicity timeseries data
mte_results_filename = 'results_day0_to_day165.pkl' #results from mTE analysis
globalIDs_filename = 'global_IDs.pkl' #global IDs from mTE analysis
localIDs_filename = 'convert_ID_day0_to_day165.pkl' #local IDs from mTE analysis

## prepare data

In [ ]:
#load toxicity network data - 

#load mTE results:
with open(datadir+mte_results_filename, 'rb') as f:
    results = pickle.load(f)

#load global IDs:
with open(datadir+globalIDs_filename, 'rb') as f:
    global_IDs = pickle.load(f)
    
#load ID conversion dict:
with open(datadir+localIDs_filename, 'rb') as f:
    convert_ID = pickle.load(f)

In [ ]:
#load resampled timeseries data - 

#resampled = pd.read_csv(datadir+'resampled.csv')
resampled = pd.read_csv(datadir+resampled_filename)

#reformat date column (str -> datetime): 
resampled['date']= pd.to_datetime(resampled['date'])

#set datetimeindex (time series data):
resampled = resampled.set_index('date').sort_index()

#pivot long into wide (sparse) format:
pivoted = pd.pivot(resampled, columns='source', values='retox')

## repack IDTxl results objects into arrays for plotting

### configuration:

In [ ]:
#input settings for sliding window (same as 4_estimate.py):
iterations = 495 #number of iterations.
window_size = 96 #samples per window.

In [ ]:
# create a template zeroed 3D array to populate with results:
x_sources = len(global_IDs) # use global ID list to determine number of sources (X).
y_targets = len(global_IDs) #use global ID list to determine number of targets (Z).
z_windows = len(results) #use number of windows to determine number of windows (Y).
template = np.zeros((x_sources, y_targets, z_windows))
print(template.shape)

### create array of significant edges:

In [ ]:
# copy template:
sig_edges_xyz = np.copy(template)

#for every window:
for r, z in zip(results, range(sig_edges_xyz.shape[-1])):
    
    # for every target present:
    for targets in r[1].get_source_variables(fdr=False):
        target = targets['target']
        
        #for every source present:
        for sourceset in targets['selected_vars_sources']:
            
            #increment edge count (significance implied):
            edge = (convert_ID[r[0]][sourceset[0]], 
                    convert_ID[r[0]][target])
            sig_edges_xyz[...,z][edge[0],edge[1]] += 1
                       
#simplify to 2D array of total edge counts over all time (-z):
sig_edges_xy = np.sum(sig_edges_xyz, axis=2)

### create array of all edges:

In [ ]:
# copy template:
tot_edges_xyz = np.copy(template)

# initialize the sliding window:
begin, end = 0, window_size

# for the stated number of iterations:
for iteration in range(iterations):
   
    # take the window; drop any chats/channels for which there is missing data (NaN)
    embedding = pivoted.iloc[begin:end].dropna(axis=1, how='any')
    sources = range(embedding.shape[1])
    
    # build a list of tuples where each pair is the global coordinates (x, y) for a considered edge
    local_coords = [c for c in list(itertools.product(sources, sources)) if len(set(c)) == len(c)]
    global_coords = [(convert_ID[str(embedding.first_valid_index())][x], convert_ID[str(embedding.first_valid_index())][y]) for x, y in local_coords]
    
    # update the array for the current z-index (iteration)
    for edge in global_coords:
        tot_edges_xyz[..., iteration][edge[0], edge[1]] += 1

    # Increment the window for the next iteration
    begin += window_size
    end += window_size

# Check if the 'end' index in the last iteration exceeds the size of 'pivoted', and adjust if necessary
if end > pivoted.shape[0]:
    end = pivoted.shape[0]

# Simplify to a 2D array by summing across the z-axis (iterations).
tot_edges_xy = np.sum(tot_edges_xyz, axis=2)

### combine arrays to obtain network data:

In [ ]:
# OPTION 1: overlay to create third array with proportion of significant edges / total edges.
#sigtot_xy = np.divide(sig_edges_xy, tot_edges_xy, out=np.zeros_like(sig_edges_xy), where=tot_edges_xy!=0)

# OPTION 2: overlay to create third array with proportion of significant edges / MAX total edges.
sigtot_xy = np.divide(sig_edges_xy, np.amax(tot_edges_xy), out=np.zeros_like(sig_edges_xy), where=tot_edges_xy!=0)

### overview of network data:

In [ ]:
#title:
print("NETWORK SUMMARY:")

# report number of nodes:
print(f"Number of nodes: {sig_edges_xy.shape[0]}")

# report number of edges:
print(f"Number of edges: {np.count_nonzero(sigtot_xy)}")

# report proportion of potential edges with at least one significant edge:
print(f"Proportion of potential edges with at least one significant edge: {np.sum(sigtot_xy)/np.count_nonzero(sigtot_xy)}")

# report max and min proportion of significant edges:
print(f"Maximum edge weight (significant/considered(max)): {np.amax(sigtot_xy)}")
print(f"Minimum (non-zero) edge weight (significant/considered(max)): {np.min(sigtot_xy[sigtot_xy>0])}")

# report average proportion of significant edges:
print(f"Average edge weight (significant/considered(max)): {np.mean(sigtot_xy)}")



### save network data:

In [ ]:
#save array to local:
np.save(datadir+'nettoxresults.npy', sigtot_xy)